In [1]:
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import numpy as np
import tensorflow as tf 
import time

pd.set_option('display.max_columns', None)
from IPython.display import display
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")
model_path  = os.path.join(pathf, "model")
log_path  = os.path.join(pathf, "model")
trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')


       x      y  z        t   terror        q  flag  event_id  hit_id
0 -142.5 -147.5  0  767.879  2.02966  1.05052     0         7       1
(9473201, 9)
   event_id  nhit  nhitreal  energymc  thetamc    phimc   xcmc    ycmc
0         7   426        70   48348.9  63.1686  11.0982 -40.83  114.03
(13315, 8)
       x      y  z        t  terror        q  event_id  hit_id
0 -142.5 -127.5  0  848.061  1.9984  1.15067         9       1
(4086511, 8)


In [3]:
#组合特征 
#train表的特征与event表的特征交互
data['fx'] = data['x']-data['xcmc']
data['fy'] = data['y']-data['ycmc']
data['fdis'] = np.sqrt(data['fx']**2+data['fy']**2)
data['fscala'] = np.sin(data['thetamc']) * data['t']
data['fphi'] = np.arctan2(data['fy'], data['fx']) * 180
data['fttrue'] = data['t']/data['terror']
data['nhitratio'] = data['nhit']/data['nhitreal']

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']

In [4]:
print(trainshape[0])
print(type(trainshape[0]))
print(data.shape)

trainpd = data[:trainshape[0]].reset_index()
testpd = data[trainshape[0]:].reset_index()
del data


9473201
<class 'int'>
(13559712, 18)


In [5]:
print(trainpd.columns)
feature= [x for x in trainpd.columns if x not in ['flag','index','hit_id','event_id']]
labels = trainpd['flag']
del trainpd['flag']
del testpd['flag']

Index(['index', 'event_id', 'flag', 'hit_id', 'q', 't', 'terror', 'nhit',
       'nhitreal', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 'fdis',
       'fscala', 'fphi', 'fttrue', 'nhitratio'],
      dtype='object')


In [6]:
# tensorflow 考虑单event聚类
def batch_iter_list(data_list, batch_size, num_epochs, shuffle=True):
    data_size = len(data_list[0])
    num_batches_per_epoch = data_size // batch_size  # 每个epoch中包含的batch数量
    for epoch in range(num_epochs):
        # 每个epoch是否进行shuflle
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data_list = [data[shuffle_indices] for data in data_list]
        else:
            shuffled_data_list = data_list

        for batch_num in range(num_batches_per_epoch + 1):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield [shuffled_data[start_index:end_index] for shuffled_data in shuffled_data_list]


In [7]:
class AbstractModeltensor(object):
    def __init__(self, config=None):
        self.config = config

    # You need to override this method.
    def buildModel(self):
        raise NotImplementedError("You need to implement your own model.")

class NeurousNet(AbstractModeltensor):
    def __init__(self, xlenth, config=None):
        super(NeurousNet, self).__init__(config)
        self.graph = tf.Graph()  # 为每个类(实例)单独创建一个graph
        self.modeldic = {
            "cnn_dense_less": self._cnn_dense_less_model,
        }
        self.ydim = 1
        self.keep_prob_ph = config["dropout"]
        self.input_dim = xlenth
        self.out_dim = 1
        with self.graph.as_default():
            with tf.name_scope('Inputs'):
                self.input_p = tf.placeholder(tf.float32, [None, self.input_dim])
                self.learn_rate_p = tf.placeholder(dtype=tf.float32, shape=[], name="lr")
                self.lr_decay = tf.placeholder(dtype=tf.float32, shape=[])
            with tf.name_scope('Outputs'):
                self.target_y = tf.placeholder(dtype=tf.float32, shape=[None, self.out_dim])

    def buildModel(self):
        tf.reset_default_graph()
        with self.graph.as_default():
            # 不同选择加载
            self.modeldic[self.config["modelname"]]()
            # 打印打包
            self.merged = tf.summary.merge_all()
            # 损失目标
            self.train_op = []
            for i2 in self.train_list:
                self.train_op.append(tf.train.AdamOptimizer(self.learn_rate_p).minimize(i2))
            # 同一保存加载
            self.saver = tf.train.Saver(tf.global_variables())
            # [print(n.name) for n in tf.get_default_graph().as_graph_def().node]
            # return self.saver

    def _cnn_dense_less_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            dense1 = tf.layers.dense(inputs=self.input_p, units=self.input_dim, activation=tf.nn.softmax, name="layer_dense1")
            tf.summary.histogram('dense1', dense1)  # 记录标量的变化
            mult_layer1 = tf.nn.softmax(dense1*self.input_p, name='mult_layer1')
            mult_layer2 = tf.nn.softmax(mult_layer1*self.input_p, name='mult_layer2')
            concat1 = tf.concat([self.input_p, dense1,mult_layer1,mult_layer2], 1, name='concat1')
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim, activation=tf.nn.elu, name="layer_dense2")
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim//4, activation=tf.nn.elu, name="layer_dense3")
            y_res = tf.layers.dense(inputs=denseo3, units=self.out_dim, activation=tf.nn.sigmoid, name="y_res")
            tf.summary.histogram('y_res', y_res)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t,0, name="y_loss_v")
            # 猜错的获取 实际盈利值的负数
            # self.learn_rate = tf.Variable(self.learn_rate_p, name="lr", trainable=False)
            # self.update_lr = tf.assign(self.learn_rate, tf.multiply(self.lr_decay, self.learn_rate))
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.histogram('mult_layer1', mult_layer1)  # 记录标量的变化
            tf.summary.histogram('mult_layer2', mult_layer2)  # 记录标量的变化
            tf.summary.histogram('y_res', y_res)  # 记录标量的变化
            
            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def batch_train_bak(self, trainpd, labels, batch_size=8, num_epochs=1, retrain=True):
        # 设置
        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if self.config["retrain"] == 1:
                    model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
                    latest_ckpt = tf.train.latest_checkpoint(model_dir)
                    if os.path.isfile("{}.index".format(latest_ckpt)):
                        self.saver.restore(sess, latest_ckpt)
                        print("retraining {}".format(latest_ckpt))
                    else:
                        sess.run(tf.global_variables_initializer())
                        print("no old model, training new----")
                writer = tf.summary.FileWriter(os.path.join(log_path, "logsevery_%s" % self.config["tailname"]),
                                               sess.graph)
                global_n = 0
                stop_n = 0
                startt = time.time()
                pre_t_base_loss = pre_t_much_loss = pre_v_much_loss = pre_v_base_loss = 100000
                
                n_splits = 5
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=4399)
                for epoch in range(num_epochs):
                    for train_index, valid_index in kf.split(trainpd):
                        inputs_t = np.array(trainpd[feature].iloc[train_index])
                        output_t = np.expand_dims(np.array(labels[train_index]),-1)
                        inputs_v = np.array(trainpd[feature].iloc[valid_index])
                        output_v = np.expand_dims(np.array(labels[valid_index]),-1)
                        dataiter = batch_iter_list([inputs_t,output_t], batch_size, num_epochs)
                        starte = time.time()
                        print("iter_trainnum", inputs_t.shape[0] // batch_size + 1)
                        for batch_num in range(inputs_t.shape[0] // batch_size + 1):
                            # 获取数据
                            r_inputs_t,r_output_t = next(dataiter)
                            feed_dict_t = {
                                self.input_p: r_inputs_t,
                                self.target_y: r_output_t,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            sess.run(self.train_op, feed_dict_t)
                            global_n += 1
                            losslist_t = sess.run(self.train_list, feed_dict_t)
                            result = sess.run(self.merged, feed_dict_t)
                            if batch_num % 20 == 0:
                                writer.add_summary(result, global_n)
                                self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                                   self.config["modelfile"]), global_step=global_n)
                                print("epocht {}, batch_num {}, step {}, time: {} s, loss_yt: {}".format(
                                        epoch, batch_num, global_n, time.time() - starte, *losslist_t))
                        # valid part
                        dataiterv = batch_iter_list([inputs_v,output_v], batch_size, num_epochs)
                        vnum_iter = inputs_v.shape[0] // batch_size + 1
                        print("iter_validnum", vnum_iter)
                        losslist_va = 0
                        for batch_num in range(vnum_iter):
                            # 获取数据
                            r_inputs_v,r_output_v = next(dataiterv)
                            feed_dict_v = {
                                self.input_p: r_inputs_v,
                                self.target_y: r_output_v,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            losslist_v = sess.run(self.valid_list, feed_dict_v)
                            losslist_va += losslist_v[0]
                        losslist_va /= vnum_iter
                        if losslist_t[0] < pre_t_base_loss and losslist_va < pre_v_base_loss:
                            stop_n += 1
                            if stop_n > self.config["early_stop"]:
                                break
                        else:
                            stop_n = 0
                        print("epochv {}, step {}, stop_n {}, time: {} s, loss_yv: {}".format(
                            epoch, global_n, stop_n, time.time() - starte, losslist_va))
                        pre_t_base_loss = losslist_t[0]
                        pre_v_base_loss = losslist_va
                writer.close()
                print("total time: %s s" % (time.time() - startt))
        # 结束
        print("train finished!")
        return None

    def batch_train(self, trainpd, labels, batch_size=8, num_epochs=1, retrain=True):
        # 设置
        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if self.config["retrain"] == 1:
                    model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
                    latest_ckpt = tf.train.latest_checkpoint(model_dir)
                    if os.path.isfile("{}.index".format(latest_ckpt)):
                        self.saver.restore(sess, latest_ckpt)
                        print("retraining {}".format(latest_ckpt))
                    else:
                        sess.run(tf.global_variables_initializer())
                        print("no old model, training new----")
                writer = tf.summary.FileWriter(os.path.join(log_path, "logsevery_%s" % self.config["tailname"]),
                                               sess.graph)
                global_n = 0
                stop_n = 0
                startt = time.time()
                pre_t_base_loss = pre_t_much_loss = pre_v_much_loss = pre_v_base_loss = 100000
                
                n_splits = 5
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=4389)
                for epoch in range(num_epochs):
                    trainevenidlist = list(set(trainpd['event_id']))
                    for train_index, valid_index in kf.split(trainevenidlist):
                        starte = time.time()
                        print("iter_trainnum", len(train_index))
                        np.random.shuffle(train_index)
                        np.random.shuffle(valid_index)
                        for batch_num, eventindex in enumerate(train_index):
                            # 获取数据
                            thisindex = trainpd[trainpd['event_id'] == trainevenidlist[eventindex]].index
                            r_inputs_t = np.array(trainpd.iloc[thisindex][feature])
                            r_output_t = np.expand_dims(np.array(labels[thisindex]),-1)
                            feed_dict_t = {
                                self.input_p: r_inputs_t,
                                self.target_y: r_output_t,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            # 更新学习率
                            sess.run(self.train_op, feed_dict_t)
                            global_n += 1
                            losslist_t = sess.run(self.train_list, feed_dict_t)
                            result = sess.run(self.merged, feed_dict_t)
                            if batch_num % 50 == 0:
                                writer.add_summary(result, global_n)
                                self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                                   self.config["modelfile"]), global_step=global_n)
                                print("epocht {}, batch_num {}, step {}, time: {} s, loss_yt: {}".format(
                                        epoch, batch_num, global_n, time.time() - starte, *losslist_t))
                        # valid part
                        print("iter_validnum", len(valid_index))
                        losslist_va = 0
                        for batch_num, eventindex in enumerate(valid_index):
                            # 获取数据
                            thisindex = trainpd[trainpd['event_id'] == trainevenidlist[eventindex]].index
                            r_inputs_v = np.array(trainpd.iloc[thisindex][feature])
                            r_output_v = np.expand_dims(np.array(labels[thisindex]),-1)
                            feed_dict_v = {
                                self.input_p: r_inputs_v,
                                self.target_y: r_output_v,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            losslist_v = sess.run(self.valid_list, feed_dict_v)
                            losslist_va += losslist_v[0]
                        losslist_va /= len(valid_index)
                        result = sess.run(self.merged, feed_dict_v)
                        writer.add_summary(result, global_n)
                        if losslist_t[0] < pre_t_base_loss and losslist_va < pre_v_base_loss:
                            stop_n += 1
                            if stop_n > self.config["early_stop"]:
                                break
                            else:
                                self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                            self.config["modelfile"]), global_step=global_n)
                        else:
                            stop_n = 0
                            self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                            self.config["modelfile"]), global_step=global_n)
                        print("epochv {}, step {}, stop_n {}, time: {} s, loss_yv: {}".format(
                            epoch, global_n, stop_n, time.time() - starte, losslist_va))
                        pre_t_base_loss = losslist_t[0]
                        pre_v_base_loss = losslist_va
                writer.close()
                print("total time: %s s" % (time.time() - startt))
        # 结束
        print("train finished!")
        return None

    def predict(self, inputs):
        model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
        print("loading model...")
        latest_ckpt = tf.train.latest_checkpoint(model_dir)

        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if os.path.isfile("{}.index".format(latest_ckpt)):
                    self.saver.restore(sess, latest_ckpt)
                else:
                    raise Exception("没有找到模型:{}".format(latest_ckpt))
                feed_dict = {
                    self.input_p: inputs,
                }
                teslist = sess.run(self.pred_list, feed_dict)
                return teslist

trainconfig ={
    "dropout":0.5,
    "tailname":"test1",
    "early_stop":100,
    "modelname":"cnn_dense_less",
    "modelfile":"v1",
    "learn_rate":0.000001,
    "retrain":1
             }
modelcrnn = NeurousNet(len(feature), config=trainconfig)
modelcrnn.buildModel()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
batch_size, num_epochs = 4096, 10000
print(trainpd.head())
globalstep = modelcrnn.batch_train(trainpd, labels, batch_size, num_epochs)

   index  event_id  hit_id          q         t   terror  nhit  nhitreal  \
0      0         7       1   1.050520  767.8790  2.02966   426        70   
1      1         7       2   0.999853  -70.5552  2.02966   426        70   
2      2         7       3   2.052540 -837.8410  1.85146   426        70   
3      3         7       4  19.513100 -973.1950  1.39994   426        70   
4      4         7       5   0.800334 -159.1400  2.02966   426        70   

   energymc  thetamc    phimc   xcmc    ycmc        fdis      fscala  \
0   48348.9  63.1686  11.0982 -40.83  114.03  280.597095  253.721415   
1   48348.9  63.1686  11.0982 -40.83  114.03  283.519540  -23.312742   
2   48348.9  63.1686  11.0982 -40.83  114.03  264.805834 -276.838153   
3   48348.9  63.1686  11.0982 -40.83  114.03  252.869393 -321.561617   
4   48348.9  63.1686  11.0982 -40.83  114.03  250.900837  -52.582798   

         fphi      fttrue  nhitratio  
0 -349.482181  378.328883   6.085714  
1 -345.372956  -34.762078   6.08

In [ ]:
y_pred = modelcrnn.predict(testpd[feature])
y_pred = np.squeeze(y_pred)

In [ ]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.5 
#生成提交文件
sub = pd.DataFrame()
sub['hit_id']=testpd['hit_id']
sub['flag_pred'] = y_pred
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()),index=False)